# Analysis of School Openspace using Maxar Vivid basemap

In [1]:
import os
import sys

import geopandas as gpd
from geopandas.tools import overlay

import shapely.geometry as geometry
from shapely.ops import linemerge, unary_union, polygonize

### _Initialise environment to create instance of WMTS-extractor client_

In [2]:
# define repo name and get root working directory
repo = 'overpass-api'
root_path = os.getcwd()[ 0 : os.getcwd().find( repo ) + len ( repo )]
root_path

'C:\\Users\\crwil\\Documents\\GitHub\\overpass-api'

In [3]:
# add repo source + notebooks directory to system path
import sys
sys.path.insert(0, os.path.join( root_path, 'src' ) )
sys.path.insert(0, os.path.join( root_path, 'notebooks' ) )

In [4]:
# add repo source + notebooks directory to system path
extractor_path = os.path.join( os.path.dirname( root_path ), 'wmts-extractor' )
sys.path.insert(0, os.path.join( extractor_path, 'src' ) )

### _Retrieve OSM school amenities intersecting arbitrary bounding box_

In [5]:
# aoi encompassing sussex county
bbox = (50.72222,-0.9575,51.16722,0.04527778)

In [6]:
# create client object
from client import Client
obj = Client()

In [7]:
# overpass query to retrieve osm amenities labelled schools
df_amenity = obj.getWaysInBoundingBox( 'amenity', bbox, value='school', tags=['name','addr:postcode'] )

In [8]:
# compute area in uk national grid coordinates
df_amenity = df_amenity.to_crs( 27700 )
df_amenity[ 'area' ] = df_amenity[ 'geometry' ].area

# sort on area column and revert to geographic coordinates
df_amenity.sort_values( 'area', ascending=False, inplace=True )
df_amenity.reset_index( drop=True, inplace=True )
df_amenity = df_amenity.to_crs( 4326 )
        
df_amenity.head(10)

,id,geometry,name,addr:postcode,area
0,314916357,"POLYGON ((-0.49428 51.15509, -0.49398 51.15495...",Cranleigh School,GU6 8QQ,516860.633414
1,168362398,"POLYGON ((-0.30112 50.84428, -0.30144 50.84428...",Lancing College,BN15 0RW,456238.455368
2,170302233,"POLYGON ((-0.94699 51.02212, -0.94602 51.02224...",Bedales Schools,GU32 2DG,425233.694465
3,227489222,"POLYGON ((-0.74335 50.84216, -0.74296 50.84227...",Westbourne House School,PO20 2BH,233501.424712
4,313763214,"POLYGON ((-0.49297 51.14893, -0.49289 51.14900...",Cranleigh Preparatory School,GU6 8QH,220862.898409
5,164938607,"POLYGON ((-0.66706 50.87637, -0.66710 50.87634...",Great Ballard School,PO18 0LR,206916.243742
6,447262338,"POLYGON ((0.00684 51.03803, 0.00708 51.03803, ...",Cumnor House School,RH17 7HT,194188.001849
7,676324975,"POLYGON ((-0.64715 51.13966, -0.64551 51.13992...",King Edward's School,GU8 5SG,193564.470110
8,26292703,"POLYGON ((-0.77348 50.83002, -0.77410 50.82829...",Chichester High School,PO19 8EB,190627.326042
9,226699636,"POLYGON ((-0.07834 50.80972, -0.08016 50.81018...",Roedean School,BN2 5RQ,186579.341272


### _Demonstrate retrieval of Sentinel-2 cloud-free mosaics collocated with largest school openspace_

In [9]:
# add columns for wmts extraction
df_amenity[ 'distance' ] = 100
df_amenity[ 'type' ] = df_amenity.iloc[ 0 ][ 'geometry' ].type
df_amenity[ 'name' ] = df_amenity.apply( lambda x: 'aoi-' + str( x.name ) , axis=1)
df_amenity.head(5)

,id,geometry,name,addr:postcode,area,distance,type
0,314916357,"POLYGON ((-0.49428 51.15509, -0.49398 51.15495...",aoi-0,GU6 8QQ,516860.633414,100,Polygon
1,168362398,"POLYGON ((-0.30112 50.84428, -0.30144 50.84428...",aoi-1,BN15 0RW,456238.455368,100,Polygon
2,170302233,"POLYGON ((-0.94699 51.02212, -0.94602 51.02224...",aoi-2,GU32 2DG,425233.694465,100,Polygon
3,227489222,"POLYGON ((-0.74335 50.84216, -0.74296 50.84227...",aoi-3,PO20 2BH,233501.424712,100,Polygon
4,313763214,"POLYGON ((-0.49297 51.14893, -0.49289 51.14900...",aoi-4,GU6 8QH,220862.898409,100,Polygon


In [10]:
import yaml
from munch import munchify
from extractor import Extractor

# load config parameters from file
cfg_path = os.path.join( extractor_path, 'cfg' )
with open( os.path.join( cfg_path, 'earthservice.yml' ), 'r' ) as f:
    config = munchify( yaml.safe_load( f ) )

In [11]:
from osgeo import gdal

import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

def plotImage( pathname, annotation, suptitle=None ):

    """
    plot image
    """
    
    def getExtent( ds ):
        
        """
        get extent
        """
    
        # get minmax image coordinates
        image_y = np.array(np.arange(0, ds.RasterYSize ) )
        image_x = np.array(np.arange(0, ds.RasterXSize ) )
        
        # get minmax map coordinates
        transform = ds.GetGeoTransform()
        map_x, map_y = ( transform[0] + image_x * transform[1] ), ( transform[3] + image_y * transform[5] )
        
        return [ np.amin( map_x ), 
                 np.amax( map_x ), 
                 np.amin( map_y ), 
                 np.amax( map_y ) ]
            
    # setup params
    plt.rc('font', size=10)   
    
    # create figure with 3857 projection
    crs = ccrs.epsg( int( 3857 ) )
    fig, ax = plt.subplots( figsize=(10,10), 
                            nrows=1, 
                            ncols=1, 
                            subplot_kw={'projection': crs },
                            constrained_layout=True )
    
    # plot suptitle if defined
    if suptitle is not None:
        fig.suptitle( suptitle, 
                    fontsize=18, 
                    horizontalalignment='center',
                    verticalalignment='center' )
    
    # open image pathname
    ds = gdal.Open( pathname )
    
    # read data
    data = ds.ReadAsArray().transpose( 1, 2, 0 )
    data = data[ ::, ::, 0:3 ]
    
    # plot image    
    ax.imshow( data, 
            extent=getExtent( ds ), 
            origin='upper', 
            interpolation='bilinear', 
            aspect='auto', 
            zorder=10 )
        
    # draw annotation
    annotation.plot(  ax=ax, 
                    facecolor='red',
                    edgecolor='black', 
                    linewidth=1,
                    alpha=0.2,
                    zorder=20)
    
    # draw gridlines
    gl = ax.gridlines( draw_labels=True,
                    linewidth=1, 
                    color='white', 
                    alpha=1.0, 
                    linestyle=':',
                    zorder=30)

    # remove labels from top and right
    gl.right_labels = gl.top_labels = False                
    return

In [12]:
# define arguments
args = munchify ( { 'zoom' : 15, 
         'root_path' : 'C:\\Users\\crwil\\Documents\\results2',
         'options' : 'TILED=YES COMPRESS=LZW',
         'overwrite' : False } )

obj = Extractor( config, args )

In [13]:
# subset largest 25 schools
df_subset = df_amenity[ 0:25 ] 
for aoi in df_subset.itertuples():

    # retrieve and plot wmts images
    pathname = obj.processAoI( aoi, None, args )   
        
    # display image + geometry annotation
    df_geom = gpd.GeoDataFrame(geometry=gpd.GeoSeries(aoi.geometry), crs=4326 )
    df_geom = df_geom.to_crs( 3857 )
        
    plotImage( pathname, df_geom, suptitle=aoi.school )

output file already exists: C:\Users\crwil\Documents\results2\aoi-0\aoi-0_15_100.TIF


AttributeError: 'Pandas' object has no attribute 'school'